### THIS FILE IS AS PER THE FEATURE SELECTION AND MODEL BUILDING STEPS THAT WERE ASKED TO DO IN THE INCLASS BY MA'AM

In [38]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline
from sklearn import metrics
import seaborn as sns

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

## LOAD THE DATASET

In [41]:
df_backpain=pd.read_csv('backpain.csv')
df_backpain

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att,Unnamed: 13
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,0.129744,7.8433,14.7484,8.51707,-15.728927,11.5472,Normal,NaN
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,0.047913,19.1986,18.1972,7.08745,6.013843,43.8693,Normal,NaN
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,0.081070,16.2059,13.5565,8.89572,3.564463,18.4151,Normal,NaN
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,0.159251,14.7334,16.0928,9.75922,5.767308,33.7192,Normal,NaN


In [42]:

df_backpain.rename(columns={"Col1":"pelvic_incidence","Col2":"pelvictilt","Col3":"lumbar_lordosis_angle",
                            "Col4":"sacral_slope","Col5":"pelvic_radius","Col6":"degree_spondylolisthesis",
                            "Col7":"pelvic_slope","Col8":"Direct_tilt","Col9":"thoracic_slope",
                            "Col10":"cervical_tilt","Col11":"sacrum_angle","Col12":"scoliosis_slope",
                            "Class_att":"Status"},inplace=True)

In [43]:
df_backpain

,pelvic_incidence,pelvictilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,pelvic_slope,Direct_tilt,thoracic_slope,cervical_tilt,sacrum_angle,scoliosis_slope,Status,Unnamed: 13
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,0.129744,7.8433,14.7484,8.51707,-15.728927,11.5472,Normal,NaN
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,0.047913,19.1986,18.1972,7.08745,6.013843,43.8693,Normal,NaN
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,0.081070,16.2059,13.5565,8.89572,3.564463,18.4151,Normal,NaN
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,0.159251,14.7334,16.0928,9.75922,5.767308,33.7192,Normal,NaN


In [44]:
df_backpain.drop('Unnamed: 13',1,inplace=True)

## Logistic regression

### A) BASE MODEL - WITH ALL THE FEATURES

In [45]:
df_backpain['Status'].replace({'Normal':'0','Abnormal':'1'},inplace=True)

### The below step is needed before building the model - because even if the ml package(sklearn) builds the model without throwing any error, we will get a value error(Pandas data cast to numpy dtype of object.) when trying to build a statistical model ,converting to integer at this step will ensure that problem doesnot arise


In [46]:
df_backpain['Status']=df_backpain['Status'].astype('int64')

In [47]:
X=df_backpain.drop(['Status'],axis=1)
y=df_backpain['Status']

In [48]:
## train test split and scaling
X_train,X_test,y_train,y_test = train_test_split(X,y.values.ravel(),test_size=0.3,random_state=7)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_scaled=pd.DataFrame(sc.fit_transform(X_train),columns=X_train.columns)
test_scaled=pd.DataFrame(sc.fit_transform(X_test),columns=X_test.columns)

In [49]:
# Model building
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression().fit(train_scaled,y_train)
y_pred=logreg.predict(X_test)

In [50]:
# checking accuracies
print("Training accuracy",logreg.score(train_scaled,y_train)) #train
print("Testing accuracy",logreg.score(test_scaled,y_test)) #test

Training accuracy 0.847926267281106
Testing accuracy 0.8279569892473119


In [51]:
logit_model=sm.Logit(y_train,train_scaled)
result=logit_model.fit()
print(result.summary2())

         Current function value: 0.504570
         Iterations: 35
                             Results: Logit
Model:                 Logit              Pseudo R-squared:   0.179     
Dependent Variable:    y                  AIC:                242.9833  
Date:                  2020-05-28 00:57   BIC:                283.5421  
No. Observations:      217                Log-Likelihood:     -109.49   
Df Model:              11                 LL-Null:            -133.31   
Df Residuals:          205                LLR p-value:        1.6554e-06
Converged:             0.0000             Scale:              1.0000    
No. Iterations:        35.0000                                          
------------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
------------------------------------------------------------------------
pelvic_incidence         -1.2058      nan     nan    nan     nan     nan
pelvictilt    

/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [52]:
## from the above summary,it is observed that there is high multicollinearity between 
#'pelvic_incidence','pelvictilt','sacral_slope'
# remove those columns and rebuild the model and check the model accuracy again


cols_to_include=[cols for cols in X.columns if cols not in ['pelvic_incidence','pelvictilt','sacral_slope']]
X=df_backpain[cols_to_include] #removing the columns

In [53]:
# rebuilding the model and checking the accuracies
X_train,X_test,y_train,y_test = train_test_split(X,y.values.ravel(),test_size=0.3,random_state=7)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_scaled=pd.DataFrame(sc.fit_transform(X_train),columns=X_train.columns)
test_scaled=pd.DataFrame(sc.fit_transform(X_test),columns=X_test.columns)

In [54]:
logreg=LogisticRegression().fit(train_scaled,y_train)
y_pred=logreg.predict(X_test)
print("training score",logreg.score(train_scaled,y_train))
print("testing score",logreg.score(test_scaled,y_test))

training score 0.8064516129032258
testing score 0.7849462365591398


In [ ]:
'''the accuracies were reduced due to the removal of the features'''

In [ ]:
## consider the below three columns and check the accuracies(based on the model summary itself)

In [55]:
X_trim_1 = X.loc[:,['lumbar_lordosis_angle','pelvic_radius','degree_spondylolisthesis']]

In [57]:
X_train,X_test,y_train,y_test = train_test_split(X_trim_1,y.values.ravel(),test_size=0.3,random_state=7)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_scaled=pd.DataFrame(sc.fit_transform(X_train),columns=X_train.columns)
test_scaled=pd.DataFrame(sc.fit_transform(X_test),columns=X_test.columns)

In [58]:
logreg=LogisticRegression().fit(train_scaled,y_train)
y_pred=logreg.predict(X_test)

print("training score",logreg.score(train_scaled,y_train))
print("testing score",logreg.score(test_scaled,y_test))

training score 0.783410138248848
testing score 0.7526881720430108


In [ ]:
'''the scores further reduced

it was observed that the confusion matrices for all the above models were giving weak accuracies,sensitivity,
specificity scores as well

Therefore ,the model building done using backward feature elimination gave best results than this kind of random
feature selection and elimination based on the model summary,for this scenario.'''